In [ ]:
#ABD Eğitim Sistemi veri seti üzerinden, notların ağırlıklı ortalamasını içeren bir değişken oluşturun. 
#4. sınıftaki öğrencilerin sayısı ile 8. sınıftaki öğrencilerin sayısı farklı olduğuna dikkat edin.

#nan değerler interpolasyon yöntemiyle doldurulur
for total in ["GRADES_4_G","GRADES_8_G","AVG_MATH_4_SCORE","AVG_MATH_8_SCORE","AVG_READING_4_SCORE","AVG_READING_8_SCORE"]:
    state_list=list(st.STATE.unique())  #tekrar etmeyen değerler
    i=0
    l=len(state_list)
    while i<l:  #her eyalet için NaN değerler doldurulur
        st[st["STATE"]==state_list[i]][total].interpolate().ffill().bfill()
        i+=1
    st=st.dropna(subset=[total])  #NaN olan satırlar bırakılır

st["AVG_MATH"]=(st["GRADES_4_G"]*st["AVG_MATH_4_SCORE"]+st["GRADES_8_G"]*st["AVG_MATH_8_SCORE"])/(st["GRADES_4_G"]+st["GRADES_8_G"])
st["AVG_READING"]=(st["GRADES_4_G"]*st["AVG_READING_4_SCORE"]+st["GRADES_8_G"]*st["AVG_READING_8_SCORE"])/(st["GRADES_4_G"]+st["GRADES_8_G"])

In [ ]:
#ABD Eğitim Sistemi veri seti üzerinden, oluşturulan ağırlıklı ortalamalara göre harcama çeşitlerinin korelasyonu nedir? 
#Hangi harcama kaleminin korelasyonu diğerlerine göre fazladır?

st_exp = st.iloc[:,9:13]  #EXP sutunları seçilir
cor1 = pd.concat([st_exp, st.AVG_MATH, st.AVG_READING], axis=1, join='inner')

plt.figure(figsize=(9,6))
sns.heatmap(cor1.corr(), square=True, annot=True, linewidths=.5, vmin=0, vmax=1, cmap='viridis')
plt.show()

In [ ]:
#ABD Eğitim Sistemi veri seti üzerinden, dört harcama kalemi için Temel Bileşenler Analizi (PCA) uygulayın! 
#Toplam varyansın ne kadarı ilk bileşen tarafından açıklanabilmektedir?

st_exp = st_exp.dropna(subset=["OTHER_EXPENDITURE"])  #NaN değerler bırakılır

pca = PCA(n_components=4)  #PCA değerleri oluşturulur
pca.fit(st_exp)
components = pca.transform(st_exp)

print(pca.explained_variance_ratio_)  #en yüksek varyans seçilir (PCA1)
print(sum(pca.explained_variance_ratio_))   #yaklaşık 1 olması beklenir
print(components.T[0])  #PCA1 değerlerini yazdırır
PCA1 = components.T[0]  #dizilerden PCA1 değerleri seçilir

In [ ]:
#ABD Eğitim Sistemi veri seti üzerinden, oluşturduğunuz genel not ortalaması ve ilk temel bileşen arasındaki korelasyon nedir?

cor1 = cor1.reset_index(drop=True)  #birleştirme için index sıfırlanır

co_exp = pd.DataFrame()
co_exp["PCA1"] = pd.DataFrame(PCA1)[0]   #PCA değerleri dataframe aktarılır
cor2 = pd.concat([co_exp, cor1], axis=1)  

plt.figure(figsize=(9,6))
sns.heatmap(cor2.corr(), square=True, annot=True, linewidths=.5, vmin=0, vmax=1, cmap='viridis')
plt.show()